In [1]:
import re
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict

import torch
from torch.nn import Embedding

from data_processing import preprocess_data

In [2]:
text_corpus = open("data/quotes.txt", "r").readlines()
text_corpus

['Let us see what love can do.\n',
 'We can’t heal the world today. But we can begin with a voice of compassion, a heart of love, and an act of kindness.\n',
 'Listen with curiosity. Speak with honesty. Act with integrity.\n',
 'The most basic and powerful way to connect to another person is to listen. Just listen.\n',
 'Knowledge speaks, but wisdom listens.\n',
 'Deep listening is the kind of listening that can help relieve the suffering of another person.\n',
 'Every person in this life has something to teach—and as soon as you accept that, you open yourself to truly listening.\n',
 'There are no problems we cannot solve together, and very few that we can solve by ourselves.\n',
 'I always wondered why somebody didn’t do something about that; then I realized that I am somebody.\n',
 'Let us develop respect for all living things. Let us try to replace violence and intolerance with understanding, compassion and love.\n',
 "We say 'things will turn out', but no. It's what we do that act

In [3]:
sentence_ids, vocab = preprocess_data(text_corpus, padding=False)

In [117]:
N = 5
ngrams_table = defaultdict(dict)

for sentence in sentence_ids:
	for i in range(len(sentence)):
		ngram = sentence[i:i+N]
		if i == 0:
			if ngram[i+1] in ngrams_table[tuple([2]*(N-1))]:
				ngrams_table[tuple([2]*(N-1))][ngram[i+1]] += 1
			else:
				ngrams_table[tuple([2]*(N-1))][ngram[i+1]] = 1
		if len(ngram) == N:
			if ngrams_table[tuple(ngram[:N-1])] == {}:
				ngrams_table[tuple(ngram[:N-1])][ngram[N-1]] = 1
			else:
				if ngram[N-1] in ngrams_table[tuple(ngram[:N-1])]:
					ngrams_table[tuple(ngram[:N-1])][ngram[N-1]] += 1
				else:
					ngrams_table[tuple(ngram[:N-1])][ngram[N-1]] = 1

print(ngrams_table)

defaultdict(<class 'dict'>, {(2, 2, 2, 2): {4: 29, 12: 151, 31: 6, 15: 523, 46: 7, 50: 4, 57: 59, 71: 108, 82: 210, 113: 171, 10: 149, 142: 34, 149: 11, 167: 1, 123: 19, 60: 180, 66: 230, 200: 235, 211: 3, 114: 39, 233: 1, 247: 19, 254: 15, 158: 3, 8: 25, 73: 58, 316: 3, 321: 4, 326: 1, 332: 33, 350: 1, 108: 190, 301: 15, 384: 1, 214: 27, 40: 84, 64: 24, 116: 14, 58: 83, 412: 10, 398: 8, 433: 1, 440: 2, 229: 2, 467: 17, 451: 6, 23: 4, 119: 15, 28: 19, 196: 16, 469: 2, 457: 22, 171: 29, 512: 5, 527: 2, 358: 17, 529: 5, 20: 12, 51: 6, 557: 5, 137: 70, 566: 4, 572: 3, 578: 3, 208: 15, 609: 2, 21: 148, 45: 31, 632: 10, 499: 8, 48: 5, 668: 2, 686: 8, 564: 5, 343: 18, 425: 19, 700: 10, 128: 25, 617: 20, 727: 1, 719: 3, 733: 6, 736: 6, 445: 3, 751: 1, 283: 7, 758: 1, 178: 15, 774: 1, 89: 2, 787: 4, 795: 2, 801: 1, 456: 5, 811: 14, 308: 12, 98: 3, 640: 4, 600: 3, 833: 4, 32: 3, 846: 2, 855: 1, 857: 1, 36: 9, 868: 25, 27: 15, 559: 4, 127: 54, 720: 1, 361: 11, 271: 16, 939: 1, 106: 3, 53: 6, 931

In [120]:
id2word = {v: k for k, v in vocab.items()}

In [ ]:
p_table = defaultdict(dict)
for k, v in ngrams_table.items():
    total_count = sum(v.values())
    p_table[k] = {word: count/total_count for word, count in v.items()}
    
# Sort the p_table for (2, 2)
sorted_p_22 = dict(sorted(p_table[(2, 2)].items(), key=lambda item: item[1], reverse=True))
print(sorted_p_22[4])

{15: 0.08663243332781183, 200: 0.03892661918171277, 66: 0.038098393241676326, 82: 0.03478548948153056, 108: 0.031472585721384796, 60: 0.02981613384131191, 113: 0.028325327149246314, 12: 0.025012423389100546, 10: 0.02468113301308597, 21: 0.02451548782507868, 71: 0.017889680304787145, 185: 0.014908066920655955, 40: 0.013914195792612225, 58: 0.013748550604604936, 137: 0.011595163160510187, 7: 0.0114295179725029, 57: 0.009773066092430015, 73: 0.009607420904422726, 127: 0.008944840152393572, 378: 0.008944840152393572, 286: 0.007950969024349842, 114: 0.006460162332284247, 142: 0.005631936392247805, 332: 0.005466291204240517, 45: 0.00513500082822594, 4: 0.004803710452211363, 171: 0.004803710452211363, 214: 0.004472420076196786, 189: 0.004472420076196786, 8: 0.004141129700182209, 128: 0.004141129700182209, 868: 0.004141129700182209, 95: 0.004141129700182209, 64: 0.003975484512174921, 330: 0.003975484512174921, 457: 0.0036441941361603446, 126: 0.0036441941361603446, 617: 0.003312903760145768, 1

In [77]:
input_seq = tuple([2]*(2))
sub_ = p_table[input_seq]

# Sample from the distribution
words = list(sub_.keys())
probabilities = list(sub_.values())
generated_word = np.random.choice(words, p=probabilities)
print(generated_word)

137


In [98]:
sub_ = p_table[(2, 3)]
sub_

{}

In [106]:
text = "if you"
input_seq = tuple(text.split()[-N+1:])
input_seq = [vocab[i] for i in input_seq]
input_seq

[200, 66]

In [111]:
keys = [key for key, val in vocab.items() if val == 66][0]
keys

'you'